In [73]:
import pandas as pd
import pickle
import itertools

from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Category10
from bokeh.layouts import gridplot
from bokeh.models import Title

output_notebook()

Loading BokehJS ...

In [2]:
def extract_scores(result):
    extracted = {}
    for kind in ("train_", "test_"):
        for score in ("AUC", "F1 Max"):
            extracted[kind + score] = result[kind + "res"][score]

    extracted.update({k:v for k, v in result.items() if not k.endswith("res")})
    return extracted  

In [23]:
results = pickle.load(open("BinaryF.p", "rb"))
scores = [extract_scores(x) for x in results]

df = pd.DataFrame(scores)
df.head()

,duration,epochs,model,sequence,size,test_AUC,test_F1 Max,train_AUC,train_F1 Max
0,9.335835,1,L5,"(3, 1)",10^5,0.453976,0.117137,0.449026,0.115465
1,7.733341,2,L5,"(3, 1)",10^5,0.513589,0.118988,0.516810,0.118044
2,7.882221,3,L5,"(3, 1)",10^5,0.571297,0.138604,0.578429,0.135557
3,7.754082,4,L5,"(3, 1)",10^5,0.718298,0.212551,0.724985,0.211710
4,8.123513,5,L5,"(3, 1)",10^5,0.746254,0.233888,0.752928,0.231837


In [25]:
model_order = ["L5", "L10_L5", "L30_L15", "L15_L10_L5", "L30_L20_L10"]
sequence_order = ["(3, 1)", "(4, 1, 3)"]

df["model"] = pd.Categorical(df["model"], model_order)
df["sequence"] = pd.Categorical(df["sequence"].astype(str), sequence_order)

[('(3, 1)', 'L5'),
 ('(3, 1)', 'L10_L5'),
 ('(3, 1)', 'L30_L15'),
 ('(3, 1)', 'L15_L10_L5'),
 ('(3, 1)', 'L30_L20_L10'),
 ('(4, 1, 3)', 'L5'),
 ('(4, 1, 3)', 'L10_L5'),
 ('(4, 1, 3)', 'L30_L15'),
 ('(4, 1, 3)', 'L15_L10_L5'),
 ('(4, 1, 3)', 'L30_L20_L10')]

In [98]:
width = 200
height = 125

def create_sub_plot(sub_df):
    p = figure(y_range=(-0.1, 1.1), width=width, height=height)

    for (size, size_df), color in zip(sub_df.groupby("size"), Category10[10]):
        series = size_df.groupby("epochs")["test_F1 Max"].mean().sort_index()
        p.line(x=series.index, y=series.values, line_color=color, legend=size)
        p.circle(x=series.index, y=series.values, line_color=color, fill_color=color, legend=size)
    
    p.yaxis.minor_tick_line_color = None
    p.xaxis.minor_tick_line_color = None
    p.xaxis.axis_label = "Epochs"
    p.yaxis.axis_label = "F1 Score"
    
    return p

In [99]:
plots = []
for idx, ((model, sequence), sub_df) in enumerate(df.groupby(["model", "sequence"])):

    p = create_sub_plot(sub_df)
    if idx % 2 == 0:
        p.add_layout(Title(text=model, align="center"), "left") 
        p.plot_width = width + 50
    else:
        p.yaxis.visible = False
        
    if idx >= 8:
        p.add_layout(Title(text=sequence, align="center"), "below")  
        p.plot_height = height + 50
    else:
        p.xaxis.visible = False
        
    if idx != 1:
        p.legend.visible = False
        
    plots.append(p)

In [100]:
show(gridplot(plots, ncols=2))